In [ ]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import calendar
import random
import seaborn as sns
import numpy as np

In [ ]:
conn = psycopg2.connect(
    port = "5432",
host = "shopify-merchant-dump.ccm9mnr5avgs.ap-south-1.rds.amazonaws.com",
  user = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c",
  password = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c_password",
  database = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c_db"
)

In [ ]:
query1 = """select o.id as order_id,
            o.created_at::date as order_date,
            o.customer_id as user_id,
            oli.product_id as product_id,
            oli.name as product_name,
            CASE
        WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
         ELSE 'Delivered'
     END as order_status,
            oli.quantity as quantity,
            oli.price as sp
            from orders o
            join order_line_item oli on oli.order_id = o.id"""

In [ ]:
data = pd.read_sql(query1, conn)

<ipython-input-4-0600e2216395>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query1, conn)


In [ ]:
data["order_date"] = pd.to_datetime(data["order_date"])

In [ ]:
data[data["product_id"] == np.nan]

,order_id,order_date,user_id,product_id,product_name,order_status,quantity,sp


In [ ]:
data['WeekCount']=data['order_date'].dt.strftime('%y-w%U')

In [ ]:
data = data.dropna()

In [ ]:
data=data.sort_values('order_date')
WeekList=data['WeekCount'].unique()

In [ ]:
WeekList.tolist()
output_list = []

for i in range(len(WeekList) - 1):
    output_list.append(WeekList[i])
    current_week = int(WeekList[i].split('-w')[1])
    next_week = int(WeekList[i + 1].split('-w')[1])
    missing_weeks = next_week - current_week - 1
    for j in range(1, missing_weeks + 1):
        missing_week = current_week + j
        output_list.append(f'{WeekList[i][:3]}w{missing_week:02d}')

output_list.append(WeekList[-1])

In [ ]:
orderuserlevel=data.groupby(['order_id','user_id']).agg(
          order_date=pd.NamedAgg(column='order_date', aggfunc='min'),

    ).reset_index()
orderuserlevel=orderuserlevel.sort_values('order_date')
orderuserlevel['TrxnRank'] = orderuserlevel.groupby('user_id')['order_date'].rank(method='first')
orderuserlevel


data=pd.merge(data,
         orderuserlevel[['order_id','TrxnRank']],
         on="order_id",
         how ='left')
data['New']=0
data['Repeat']=0
data.loc[data['TrxnRank']==1,'New']=1
data.loc[data['TrxnRank']>1,'Repeat']=1

#AOV Contribution
data['ordervalue']=data['quantity']*data['sp']
data['AOVContribution']=data['ordervalue']/data["order_id"].nunique()
data['DeltaDaysinOrders']=(data.groupby('user_id').order_date.shift() - data.order_date).dt.days.abs()
data['DeltaOrderValues']=(data.groupby('user_id').ordervalue.shift() - data.ordervalue)

In [ ]:
weeklyusersegsats = {}
for x in output_list:
  x3 = data[data['WeekCount'] <= x] #50
  try:
    userweeklydata = x3.groupby("user_id").agg(
          Last_Date=pd.NamedAgg(column='order_date', aggfunc= lambda x: x.nlargest(2).min()),
          First_Date=pd.NamedAgg(column='order_date', aggfunc='min'),
          OrdersCount=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
          Mean_Days_Delta=pd.NamedAgg(column='DeltaDaysinOrders', aggfunc='mean'),
          Mean_OV_Delta=pd.NamedAgg(column='DeltaOrderValues', aggfunc='mean')
          ).reset_index()
    globalmaxdateweek=x3['order_date'].max()
    userweeklydata['Recency']=globalmaxdateweek-userweeklydata['Last_Date']
    userweeklydata['Recency']=userweeklydata['Recency'].dt.days.abs()
    userweeklydata['Activated_Quater']=userweeklydata['First_Date'].dt.to_period('Q')
    userweeklydata['Maturity_Age']=userweeklydata['Last_Date']-userweeklydata['First_Date']
    userweeklydata['Maturity_Age']=userweeklydata['Maturity_Age'].dt.days.abs()
    userweeklydata['Mean_Days_Delta']=userweeklydata['Mean_Days_Delta'].fillna(0)
    userweeklydata['Mean_OV_Delta']=userweeklydata['Mean_OV_Delta'].fillna(0)
    userweeklydata['Customer_AOV']=userweeklydata['Order_Value']/userweeklydata['OrdersCount']

    r_labels, f_labels,AOV_labels, m_labels,DelAOV_labels = range(4, 0, -1), range(1,5), range(1,5),range(1,5),range(1,5)
    userweeklydata['f_score'] = userweeklydata['OrdersCount'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = f_labels)).astype('int64')
    # userweeklydata['f_score'] = userweeklydata['OrdersCount'].transform(lambda x: pd.qcut(x[~x.duplicated()].rank(method='first'), q=[0, 0.25, 0.5, 0.75, 1], labels=f_labels)).astype('int64')
    userweeklydata['r_score'] = userweeklydata['Recency'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = r_labels)).astype('int64')
    userweeklydata['AOV_score'] = userweeklydata['Customer_AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = AOV_labels)).astype('int64')
    userweeklydata['m_score'] = userweeklydata['Maturity_Age'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = m_labels)).astype('int64')
    userweeklydata['DelAOV'] = userweeklydata['Mean_OV_Delta'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = DelAOV_labels)).astype('int64')
    userweeklydata['SumScore'] = userweeklydata['r_score'] + userweeklydata['m_score'] + userweeklydata['AOV_score']+ userweeklydata['DelAOV'] +userweeklydata['f_score']
    def CohortLables(df):
        if df['SumScore'] >= 18:
            return 'Champions'
        elif ((df['m_score'] <=1) and (df['SumScore'] >= 10)and (df['SumScore'] < 18) and (df['r_score'] >1)):
            return 'New&Potential'
        elif ((df['m_score'] >1) and (df['SumScore'] >= 14)and (df['SumScore'] < 18)and (df['r_score'] >1)):
            return 'Mature&Loyal'
        elif ((df['m_score'] >1) and (df['SumScore'] >= 10)and (df['SumScore'] < 14)and (df['r_score'] >1)):
            return 'Mature&Slow'
        elif ((df['m_score'] <=1) and (df['SumScore'] >= 5)and (df['SumScore'] < 10)and (df['r_score'] >1)):
            return 'New&Slow'
        elif ((df['m_score'] >1) and (df['SumScore'] >= 5)and (df['SumScore'] < 10)and (df['r_score'] >1)):
            return 'Mature&ChurnRisk'
        else:
            return 'Churned'

    userweeklydata['CohortLabel'] = userweeklydata.apply(CohortLables, axis=1)

    usersegstats=userweeklydata.groupby(
      'CohortLabel'
    ).agg(
          Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
          Orders=pd.NamedAgg(column='OrdersCount', aggfunc='sum'),
          Recency=pd.NamedAgg(column='Recency', aggfunc='mean'),
          OrderValue=pd.NamedAgg(column='Order_Value', aggfunc='sum'),
          MeanDaysDelta=pd.NamedAgg(column='Mean_Days_Delta', aggfunc='mean'),
          MeanOVDelta=pd.NamedAgg(column='Mean_OV_Delta', aggfunc='mean'),
          Maturity=pd.NamedAgg(column='Maturity_Age', aggfunc='mean'),
    )

    usersegstats["Frequency"] = usersegstats["Orders"] / usersegstats["Users"]
    usersegstats["AOVContribution"] = usersegstats["OrderValue"] / usersegstats["Orders"].sum()
    usersegstats["AOV"] = usersegstats["OrderValue"] / usersegstats["Orders"]
    user_cohort_labels = ['Champions', 'Churned', 'Mature&ChurnRisk', 'Mature&Loyal', 'Mature&Slow', 'New&Potential', 'New&Slow']
    var_list = ['Users', 'Orders', 'Recency', 'AOVContribution', 'OrderValue',
        'MeanDaysDelta', 'MeanOVDelta', 'Maturity', 'Frequency', 'AOV']
    dummy_df = pd.DataFrame(index=user_cohort_labels, columns=var_list)
    for i in user_cohort_labels:
        try:
          dummy_df.loc[i] = usersegstats.loc[i]
        except:
          pass
    dummy_df = dummy_df.fillna(0)

      # usersegstats = usersegstats.reset_index().iloc[:, 1:]
    usersegstats = dummy_df
    usersegstats = usersegstats.stack()
    usersegstats=usersegstats.reset_index()
    usersegstats = usersegstats.rename(columns={'level_0': 'User Cohort Lables'})
    usersegstats=usersegstats.rename(columns={"level_1": "Product Variable"})
    usersegstats=usersegstats.rename(columns={0: "Product Value"})
    weeklyusersegsats[x] = usersegstats
  except:
    pass

In [ ]:
each_metric_df = {}
ListofCustomerCohortVariables = usersegstats.columns.to_list()
Line_Chart_Variable_List_Customer = ['Recency', 'OrderValue', 'MeanDaysDelta', 'MeanOVDelta', 'Maturity', 'Frequency', 'AOV']
Stack_Chart_Variable_List_Customer = ['Users', 'Orders', 'AOVContribution']
for week, df in weeklyusersegsats.items():
    df.sort_index(inplace=True)

def metric_graph(metric):
  if metric in Line_Chart_Variable_List_Customer:
    column_df = pd.DataFrame({week: df[metric].tolist() for week, df in weeklyusersegsats.items()})
    column_df.index = user_cohort_labels
    column_transposed = column_df.transpose()
    insights_table = column_transposed
    each_metric_df[metric] = column_transposed
    weeks = column_transposed.index.to_list()
    for column_name, values in column_transposed.items():
      plt.plot(weeks, values, label=column_name)
    plt.xlabel('Weeks')
    plt.ylabel(metric)
    plt.title(f'Weekly {metric}')
    plt.xticks(fontsize=4, rotation=90)
    plt.legend(loc='upper left', bbox_to_anchor=(1,1), title = 'Customer Cohort Lables')
    plt.show()

  if metric in Stack_Chart_Variable_List_Customer:
    variable_df = pd.DataFrame({week: df[metric].tolist() for week, df in weeklyusersegsats.items()})
    variable_df.index = user_cohort_labels
    variable_df_transposed = variable_df.transpose()
    each_metric_df[metric] = variable_df_transposed
    insights_table = variable_df_transposed
    graph2 = variable_df_transposed
    graph2.plot(kind='bar', stacked=True, figsize=(8, 4))
    plt.title(f'Weekly {metric}')
    plt.xticks(fontsize=4)
    plt.xlabel('Weeks')
    plt.ylabel(metric)
    plt.legend(loc='upper left', bbox_to_anchor=(1,1), title = 'Customer Cohort Lables')
    plt.show()

output = metric_graph('Recency')
output

KeyError: ignored

In [ ]:
def metric_cohort_graph(metric, cohort):
    if metric in Line_Chart_Variable_List_Customer:
        column_df = pd.DataFrame({week: df[metric].tolist() for week, df in weeklyusersegsats.items()})
        column_df.index = user_cohort_labels
        column_transposed = column_df.transpose()
        each_metric_df[metric] = column_transposed[cohort]  # Select only the specified cohort column

        # Replace zero values with np.nan
        each_metric_df[metric] = np.where(each_metric_df[metric] == 0, np.nan, each_metric_df[metric])

        plt.plot(column_transposed.index, each_metric_df[metric], label=cohort)
        plt.xlabel('Weeks')
        plt.ylabel(metric)
        plt.title(f'Weekly {metric}')
        plt.xticks(fontsize=4, rotation=90)
        plt.legend(loc='upper left', bbox_to_anchor=(1, 1), title='Customer Cohort Labels')
        plt.show()

    if metric in Stack_Chart_Variable_List_Customer:
        variable_df = pd.DataFrame({week: df[metric].tolist() for week, df in weeklyusersegsats.items()})
        variable_df.index = user_cohort_labels
        variable_df_transposed = variable_df.transpose()
        each_metric_df[metric] = variable_df_transposed[cohort]  # Select only the specified cohort column

        # Replace zero values with np.nan
        each_metric_df[metric] = np.where(each_metric_df[metric] == 0, np.nan, each_metric_df[metric])

        graph2 = pd.DataFrame({cohort: each_metric_df[metric]})
        graph2.plot(kind='bar', stacked=True, figsize=(8, 4))
        plt.title(f'Weekly {metric}')
        plt.xticks(fontsize=4)
        plt.xlabel('Weeks')
        plt.ylabel(metric)
        plt.legend(loc='upper left', bbox_to_anchor=(1, 1), title='Customer Cohort Labels')
        plt.show()

# Call the function with the specified metric and cohort
result = metric_cohort_graph('Recency', 'Champions')